In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
!pip install omegaconf

Defaulting to user installation because normal site-packages is not writeable


In [40]:
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [41]:
import logging
logging.basicConfig(level=logging.INFO)

from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

In [42]:
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service
INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


In [44]:
# configuration: what to download
cfg_dict = dict(
    tile_dir='/media/user/repos/bulk_euclid_cutouts/tests/test_downloading',
    release_name='F-006',
    bands=['VIS', 'NIR_Y'],
    auxillary_products=['MERPSF', 'MERRMS', 'MERBKG']
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc

In [45]:
tile_index = 102163935  # in the full pipeline, we iterate through each tile index, so it isn't part of cfg

In [46]:
flux_tile_metadata = pipeline_utils.get_tiles_in_survey(tile_index=tile_index, bands=cfg.bands, release_name=cfg.release_name)
flux_tile_metadata

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results


,category,checksum,creation_date,crpix1,crpix2,crval1,crval2,data_release_oid,data_size,dec,...,ra,release_date,release_name,second_type,stc_s,technique,tile_index,to_be_published,zero_point,zero_point_error
0,SCIENCE,b337648ff610da4e1325eb508cdd0b5e,2024-05-18T15:25:47.352,9600.0,9720.0,191.7086,75.99535,9,-32.0,75.99535,...,191.7086,<NA>,F-006,SKY,Polygon J2000 192.78985276363713 75.7229415081...,IMAGE,102163935,1,24.6,0.1
1,SCIENCE,c374e54fbca6371ef8dd3299e5d0ccf3,2024-05-18T16:02:32.437,9600.0,9720.0,191.7086,75.99535,9,-32.0,75.99535,...,191.7086,<NA>,F-006,SKY,Polygon J2000 192.78985276363713 75.7229415081...,IMAGE,102163935,1,29.8,0.1


In [47]:
flux_tile = flux_tile_metadata.iloc[1]
auxillary_tile_metadata = pipeline_utils.get_auxillary_tiles(flux_tile['mosaic_product_oid']) 
auxillary_tile_metadata

,aux_mosaic_oid,checksum,file_name,file_path,mosaic_product_oid,product_type_sas,stype,to_be_published
0,53097,988b6afc43269b76e41e0f101e22ccbe,EUC_MER_BGMOD-NIR-Y_TILE102163935-2E43F7_20240...,/data_staging_otf/repository_otf/F-006/MER/102...,11820,MERBKG,BKG,1
1,53100,9223feedeb7ccad1733cd1177b0571dd,EUC_MER_MOSAIC-NIR-Y-RMS_TILE102163935-2EFA1E_...,/data_staging_otf/repository_otf/F-006/MER/102...,11820,MERRMS,RMS,1
2,53099,933c204780826b343bbd0930812ef507,EUC_MER_CATALOG-PSF-NIR-Y_TILE102163935-3B0B0B...,/data_staging_otf/repository_otf/F-006/MER/102...,11820,MERPSF,PSF,1


In [48]:
pipeline.download_all_data_at_tile_index(cfg, tile_index)

INFO:astroquery:Query finished.
INFO:root:Downloaded flux tiles: {'VIS': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-VIS_TILE102163935-1FAFF8_20240517T094133.082498Z_00.00.fits', 'NIR_Y': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102163935-A346C5_20240517T041910.592233Z_00.00.fits'}


INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results


INFO:root:Downloaded flux+auxillary tiles: {'VIS': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-VIS_TILE102163935-1FAFF8_20240517T094133.082498Z_00.00.fits', 'NIR_Y': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102163935-A346C5_20240517T041910.592233Z_00.00.fits', 'MERBKG': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGMOD-NIR-Y_TILE102163935-2E43F7_20240517T041910.592020Z_00.00.fits', 'MERPSF': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_CATALOG-PSF-NIR-Y_TILE102163935-3B0B0B_20240517T133139.631245Z_00.00.fits', 'MERRMS': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_MOSAIC-NIR-Y-RMS_TILE102163935-2EFA1E_20240517T034329.535179Z_00.00.fits'}


{'VIS': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-VIS_TILE102163935-1FAFF8_20240517T094133.082498Z_00.00.fits',
 'NIR_Y': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102163935-A346C5_20240517T041910.592233Z_00.00.fits',
 'MERBKG': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_BGMOD-NIR-Y_TILE102163935-2E43F7_20240517T041910.592020Z_00.00.fits',
 'MERPSF': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_CATALOG-PSF-NIR-Y_TILE102163935-3B0B0B_20240517T133139.631245Z_00.00.fits',
 'MERRMS': '/media/user/repos/bulk_euclid_cutouts/tests/test_downloading/EUC_MER_MOSAIC-NIR-Y-RMS_TILE102163935-2EFA1E_20240517T034329.535179Z_00.00.fits'}